### Upload files to AWS S3

In [1]:
import boto3

s3 = boto3.client('s3')

bucket_name= 'mypracawsbucketsc2'
file_path ='yellow_tripdata_2015-01.csv'
s3_key= 'raw/yellow_tripdata_2015-01.csv'

s3.upload_file(file_path, bucket_name, s3_key)
print("Upload completed.")

Upload completed.


### Create connection to AWS S3

In [2]:
import os
import findspark

# Ustawienie paczek potrzebnych do obsługi S3 (muszą być przed findspark.init())
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages org.apache.hadoop:hadoop-aws:3.3.1,"
    "com.amazonaws:aws-java-sdk-bundle:1.11.375,"
    "org.postgresql:postgresql:42.2.27 pyspark-shell"
)

findspark.init()

from pyspark.sql import SparkSession

def create_spark_session(app_name="S3 CSV Reader"):
    # Słownik konfiguracji Sparka dla S3
    spark_conf = {
        "spark.jars.packages": "org.apache.hadoop:hadoop-aws:3.3.1,"
                              "com.amazonaws:aws-java-sdk-bundle:1.11.375",
        "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
        "spark.hadoop.fs.s3a.aws.credentials.provider": "com.amazonaws.auth.DefaultAWSCredentialsProviderChain",
        # dodatkowe opcje można dorzucić tu, np. endpoint, ssl itp.
    }

    builder = SparkSession.builder.appName(app_name)
    for k, v in spark_conf.items():
        builder = builder.config(k, v)
    
    spark = builder.getOrCreate()
    return spark

spark = create_spark_session()

25/07/01 14:50:55 WARN Utils: Your hostname, DESKTOP-SI2ABOJ resolves to a loopback address: 127.0.1.1; using 172.18.207.38 instead (on interface eth0)
25/07/01 14:50:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/pbartosz/Programowanie/DataEngineering/venv/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pbartosz/.ivy2/cache
The jars for the packages stored in: /home/pbartosz/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9a6bfed0-bb33-442d-98bc-e0815b5c4ed0;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.postgresql#postgresql;42.2.27 in central
	found org.checkerframework#checker-qual;3.5.0 in central
downloading https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.27/postgresql-42.2.27.jar ...
	[SUCCESSFUL ] org.postgresql#postgresql;42.2.27!postgresql.jar (256ms)
downloading https://repo1.maven.org/maven2/org/checkerframework/checker-qual/3.5.0/checker-qual-3.5.0.jar ...
	[SUCCESSFUL ] org.checkerfr

### Read CSV file

In [3]:
df = spark.read.csv("s3a://mypracawsbucketsc2/raw/yellow_tripdata_2015-01.csv", header=True, inferSchema=True)


25/07/01 14:51:31 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


### Transform

In [4]:
from pyspark.sql.functions import col, to_timestamp

df_clean = df.withColumn("tpep_pickup_datetime", to_timestamp(col("tpep_pickup_datetime"))) \
            .withColumn("tpep_dropoff_datetime", to_timestamp(col("tpep_dropoff_datetime"))) \
            .dropna()

### Agregate some data

In [5]:
agg_df = df_clean.groupBy("VendorID").avg("total_amount")

### Save as parquet to S3


In [6]:
agg_df.write.mode("overwrite").parquet("s3a://mypracawsbucketsc2/processed/avg_amount_per_vendor.parquet")